In [2]:
import logging
import os
from datetime import datetime


In [3]:
datetime.now()

datetime.datetime(2024, 2, 28, 17, 1, 47, 335850)

In [4]:
f"{datetime.now().strftime('%m_%d_%Y_%H_%M_%S')}.log"

'02_28_2024_17_01_47.log'

In [5]:
os.path.join(os.getcwd(),"logs")

'c:\\FSDS\\notebooks\\logs'

In [6]:
from src.DimondPricePrediction.logger import logging

In [7]:
logging.info("I have just tested the things")

In [8]:
import sys

In [9]:
try:
        a=1/0
except Exception as e:
        print(e)

division by zero


In [10]:
try:
        a=1/0
except Exception as e:
        _,_,exc_tb = sys.exc_info()
        print(exc_tb.tb_lineno)
        print(exc_tb.tb_frame.f_code.co_filename)
        print(sys.exc_info())
        

2
C:\Users\HP\AppData\Local\Temp\ipykernel_21288\3930040423.py
(<class 'ZeroDivisionError'>, ZeroDivisionError('division by zero'), <traceback object at 0x00000290C5306300>)


In [11]:
os.makedirs(os.path.dirname(os.path.join("artifacts","raw.csv")),exist_ok =True)

In [12]:
X_train,y_train,X_test,y_test =(
                train_array[:,:-1],
                 train_array[:,-1],
                  test_array[:,:-1],
                   test_array[:,-1]
     )

NameError: name 'train_array' is not defined

In [14]:
from src.DimondPricePrediction.utils.utils import save_object

ImportError: cannot import name 'save_object' from 'src.DimondPricePrediction.utils.utils' (c:\fsds\src\DimondPricePrediction\utils\utils.py)

In [13]:
def save_obj(file_path,obj):
    try:
        dir_path = os.path.dirname(file_path)

        os.makedirs(dir_path,exist_ok=True)

        with open(file_path,"wb") as file_obj:
            pickle.dump(obj,file_obj)

    except Exception as e:
        raise customexception(e,sys)    

Artifacts and configuration
Artifacts are output of each component like data_ingestion--->Raw data 
data_trans---> preprocessed object
model_trainer---> model object

configuration in present inside each component module


In [ ]:
def initialize_data_transformation(self, train_path, test_path):
        try:
            train_df=pd.read_csv(train_path)
            test_df=pd.read_csv(test_path)
            logging.info("read train and test data complete")
            logging.info(f'Train DataFrame Head : \n {train_df.head().to_string()}')
            raise customexception(e,sys)                            
        except Exception as e:
            logging.info("Exception occours in the initiate_datatransformation")
            raise customexception(e,sys)

In [15]:
def initialize_data_transformation(self, train_path, test_path):
        try:
            train_df=pd.read_csv(train_path)
            test_df=pd.read_csv(test_path)
            logging.info("read train and test data complete")
            logging.info(f'Train DataFrame Head : \n{train_df.head().to_string()}')
            logging.info(f'Test DataFrame Head : \n{test_df.head().to_string()}')


            preprocessing_obj= self.get_data_transformation()  

            #segregate coloumns
            target_column_name ='price' 
            drop_columns=[target_column_name,'id']

            input_feature_train_df = train_df.drop(columns=drop_columns,axis=1)
            input_feature_test_df = test_df.drop(columns=drop_columns,axis=1)

            target_feature_train_df = train_df[target_column_name]
            target_feature_test_df=  test_df[target_column_name]

            input_feature_train_arr = preprocessing_obj.fit_transform(input_feature_train_df)
            input_feature_test_arr = preprocessing_obj.transform(input_feature_test_df)

            logging.info("Applying preprocessing object on train and test datasets")

            train_arr= np.c_[input_feature_train_arr,np.array(target_feature_train_df)]
            test_arr= np.c_[input_feature_test_arr,np.array(target_feature_test_df)]

            
            save_object(
                file_path=self.data_transformation_config.preprocessor_obj_file_path,
                obj =preprocessing_obj
            )

            logging.info("Preprocessing pickle file saved")

            return (
                train_arr,
                test_arr
            )


        except Exception as e:
            logging.info("Exception occours in the initiate_datatransformation")
            raise customexception(e,sys)

In [16]:
import pandas as pd
import numpy as np
from src.DimondPricePrediction.logger import logging
from src.DimondPricePrediction.exception import customexception
import os
import sys
from dataclasses import dataclass
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OrdinalEncoder,StandardScaler
from src.DimondPricePrediction.utils.utils import save_object

@dataclass
class DataTransformationConfig:
    preprocessor_obj_file_path =os.path.join('artifacts','preprocessor.pkl')

class DataTransformation: 
    def __init__(self):
        self.data_transformation_config =DataTransformationConfig()

    def get_data_transformation(self):
        try:
            logging.info("Data Transformation initiated")
            #define which coloumn should be ordinal-encoded and which should be scaled.
            categorical_cols = ['cut', 'color', 'clarity']
            numerical_cols= ['carat', 'depth', 'table', 'x', 'y', 'z']
            
            # Define the custom ranking for each ordinal variable
            cut_categories=[ 'Fair','Good','Very Good','Premium','Ideal']
            clarity_categories=[ 'I1','SI2','SI1','VS2', 'VS1','VVS2','VVS1', 'IF'  ]
            color_categories=['D','E','F','G','H','I','J']
            logging.info("pipeline initiated")


            #pipelines for transformation
            ##Numerical pipeline
            num_pipeline = Pipeline(
                    steps=[
                        ('imputer', SimpleImputer(strategy='median')),
                        ('scaler',StandardScaler())
                    ]

            )
            ##Categorical Pipeline
            cat_pipeline = Pipeline(
                    steps=[
                        ('imputer', SimpleImputer(strategy='most_frequent')),
                        ('ordinalencoder',OrdinalEncoder(categories=[cut_categories,clarity_categories,color_categories])),
                        ('scaler',StandardScaler())

                    ]
            )
            #transformation object
            preprocessor=ColumnTransformer([
                ('num_pipeline',num_pipeline,numerical_cols),
                ('cat_pipeline',cat_pipeline,categorical_cols)

            ])

            return preprocessor


        except Exception as e:
            logging.info("Exception occours in the initiate_datatransformation")
            raise customexception(e,sys)


    def initialize_data_transformation(self, train_path, test_path):
        try:
            train_df=pd.read_csv(train_path)
            test_df=pd.read_csv(test_path)
            logging.info("read train and test data complete")
            logging.info(f'Train DataFrame Head : \n{train_df.head().to_string()}')
            logging.info(f'Test DataFrame Head : \n{test_df.head().to_string()}')


            preprocessing_obj= self.get_data_transformation()  

            #segregate coloumns
            target_column_name ='price' 
            drop_columns=[target_column_name,'id']

            input_feature_train_df = train_df.drop(columns=drop_columns,axis=1)
            input_feature_test_df = test_df.drop(columns=drop_columns,axis=1)

            target_feature_train_df = train_df[target_column_name]
            target_feature_test_df=  test_df[target_column_name]

            input_feature_train_arr = preprocessing_obj.fit_transform(input_feature_train_df)
            input_feature_test_arr = preprocessing_obj.transform(input_feature_test_df)

            logging.info("Applying preprocessing object on train and test datasets")

            train_arr= np.c_[input_feature_train_arr,np.array(target_feature_train_df)]
            test_arr= np.c_[input_feature_test_arr,np.array(target_feature_test_df)]

            
            save_object(
                file_path=self.data_transformation_config.preprocessor_obj_file_path,
                obj =preprocessing_obj
            )

            logging.info("Preprocessing pickle file saved")

            return (
                train_arr,
                test_arr
            )


        except Exception as e:
            logging.info("Exception occours in the initiate_datatransformation")
            raise customexception(e,sys)

ImportError: cannot import name 'save_object' from 'src.DimondPricePrediction.utils.utils' (c:\fsds\src\DimondPricePrediction\utils\utils.py)